In [1]:
import numpy as np
import pandas as pd
import glob
import random
import scipy.stats

In [2]:
from geopy import distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print (distance.geodesic(coords_1, coords_2).km)

279.35290160430094


In [3]:
trippath =r'C:\Users\User\Desktop\Data\Trainingdata\tripsample'
# trippath = r'C:\Users\User\Desktop\Data\test\trip'
Trip_file = glob.glob(trippath + "/*.txt")
logpath =r'C:\Users\User\Desktop\Data\Trainingdata\logsample'
# logpath = r'C:\Users\User\Desktop\Data\test\log'
Log_file = glob.glob(logpath + "/*.txt")

In [4]:
lon_range = [103.6, 104.042]
lat_range = [1.238, 1.48]

In [5]:
def preprocess(df):
    print("df shape before Lat Lon process:")
    print(df.shape)
    
    ## Lat Lon ##
    df = df[(df['Lat'] >= lat_range[0]) & (df['Lat'] <= lat_range[1])]
    df = df[(df['Lon'] >= lon_range[0]) & (df['Lat'] <= lon_range[1])]
    print("df shape after Lat Lon process:")
    print(df.shape)    
    
    
    ## Time ##
    df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d %X")
    df.sort_values(by='Time', inplace=True, ascending=True)

    df['Timedif'] = df['Time'] - df['Time'].shift(1)
    df['Timedif'] = df['Timedif'].astype(np.int64) // 10 ** 9.
    df = df.drop(['Time'], axis=1)
    df = df.reset_index(drop=True)
    df.iloc[0]['Timedif'] = 0
    df.iloc[0]['Timedif'] = np.mean(df['Timedif'])

    
    ## Distance ##
    coords = (df['Lat'], df['Lon'])
#     print(coords)
    lat = coords[0]
    lon = coords[1]
    distdif = []
    for i in range(0, len(lat) - 1):
#         print(distance.geodesic((lat[i], lon[i]), (lat[i + 1], lon[i + 1])).km)
        distdif.append(distance.geodesic((lat[i], lon[i]), (lat[i + 1], lon[i + 1])).km)
    mean = np.mean(distdif)
    distdif.insert(0, mean)
    df['Distdif'] = distdif
    
    
    ## Speed ##
    speed = []
    timedif = df['Timedif']
    for i in range(1, len(timedif)):
        s = distdif[i] / timedif[i]
        if s <= 0.0333:
            speed.append(s)
        else:
            speed.append(0.0333)
#         speed.append(distdif[i] / timedif[i]) # km/s
    mean = np.mean(speed)
    speed.insert(0, mean)
    df['Speed'] = speed
    
    
    ## Flag ## - 2 diff source will return 1, same source returns 0
    flag = []
    source = df['Source']
    flag.append(0) # adding 0 to the first value
    for i in range(len(source) - 1):
        if source[i] != source[i + 1]:
            flag.append(1)
        else:
            flag.append(0)
    df['Flag'] = flag    
    
    
    ## Drop first row ##
    df = df.drop([0])
    print("df shape final:")
    print(df.shape)
    
    return df

In [6]:
## Handling inf/-inf/NaN/null values + values at the extremities ##
def dropna(df):
    ## Check if any index has a null value ##
    print(df.notnull().values.all())
    print(df.shape)
    
    subset = ["Speed", "Distdif", "Timedif", "Lat", "Lon"]
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=subset, how="any")
    print(df.shape)
    
    return df

In [12]:
def entropy(series):
    p_data = series.value_counts()
    entropy = scipy.stats.entropy(p_data)
    return entropy

In [13]:
header = ["Mean", "Stdev", "Over60", "O60U120", "Over120", "Skewness", "Kurtosis", "Entropy", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9"]

def further_process(df):
    out = pd.DataFrame(columns=header)
    val = []
    
    df = dropna(df) #filter away all nan
#     df = df[df["Timedif"] <= 1440] #filter for time <= 24 mins
    df = df[df["Flag"] == 1] #filter for flag == 1
    print(df.shape)
    
    total = len(df)
    speed = df["Speed"]
    npArr = speed.values
    
    ## Mean, std ##
    mean = np.mean(npArr)
    std = np.std(npArr)
    val.append(mean)
    val.append(std)
    
    ## % over ##
    mask120 = (speed >= 0.0333)
    mask60 = (speed >= 0.01666)
    count120 = len(df[mask120])
    count60 = len(df[mask60])
    over60 = count60 / total
    over60under120 = (count60 - count120) / total
    over120 = count120 / total
    val.append(over60)
    val.append(over60under120)
    val.append(over120)
    print(over120, over60, total)
    
    ## Skewness ##
    val.append(speed.skew(axis=0))
    
    ## Kurtosis ##
    val.append(speed.kurtosis())
    
    ## Entropy ##
    val.append(entropy(speed))
    
    ## Quantile ##
    q1 = speed.quantile(.1)
    q2 = speed.quantile(.2)
    q3 = speed.quantile(.3)
    q4 = speed.quantile(.4)
    q5 = speed.quantile(.5)
    q6 = speed.quantile(.6)
    q7 = speed.quantile(.7)
    q8 = speed.quantile(.8)
    q9 = speed.quantile(.9)
    val.append(q1)
    val.append(q2)
    val.append(q3)
    val.append(q4)
    val.append(q5)
    val.append(q6)
    val.append(q7)
    val.append(q8)
    val.append(q9)

    out.loc[0] = val
#     return pd.DataFrame(data=val, dtype=np.float64)
    return out

In [14]:
labels = pd.DataFrame()
records = pd.DataFrame()

index = 0
while index < len(Trip_file):
    ## same ##
    df1 = pd.read_csv(Trip_file[index], usecols=[1, 2, 3, 4], sep=' ')
    df2 = pd.read_csv(Log_file[index], usecols=[1,2,3,4], sep=' ')
    out = preprocess(pd.concat([df1, df2]))
    out = further_process(out)
    records = pd.concat([records, out], ignore_index=True)
    labels = pd.concat([labels, pd.DataFrame(data=[1])], ignore_index=True)
    
    ## different ##
    rand = random.randint(0, len(Trip_file) - 1)
    while rand == index:
        rand = random.randint(0, len(Trip_file) - 1)
    df3 = pd.read_csv(Log_file[rand], usecols=[1,2,3,4], sep=' ')
    out = preprocess(pd.concat([df1, df3]))
    out = further_process(out)
    records = pd.concat([records, out], ignore_index=True)
    labels = pd.concat([labels, pd.DataFrame(data=[0])], ignore_index=True)
    index = index + 1

df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(598, 4)


C:\Users\User\AppData\Local\Continuum\anaconda3\envs\conda37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


df shape final:
(597, 7)
True
(597, 7)
(597, 7)
(340, 7)
0.0 0.0029411764705882353 340
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(599, 4)


C:\Users\User\AppData\Local\Continuum\anaconda3\envs\conda37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


df shape final:
(598, 7)
True
(598, 7)
(598, 7)
(290, 7)
0.1 0.16551724137931034 290
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(597, 4)


...

C:\Users\User\AppData\Local\Continuum\anaconda3\envs\conda37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\User\AppData\Local\Continuum\anaconda3\envs\conda37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars


df shape final:
(599, 7)
True
(599, 7)
(599, 7)
(263, 7)
0.060836501901140684 0.12167300380228137 263
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(599, 4)
df shape final:
(598, 7)
True
(598, 7)
(598, 7)
(424, 7)
0.0 0.007075471698113208 424
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(599, 4)
df shape final:
(598, 7)
True
(598, 7)
(598, 7)
(354, 7)
0.06497175141242938 0.12429378531073447 354
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
True
(599, 7)
(599, 7)
(371, 7)
0.0 0.01078167115902965 371
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(598, 4)
df shape final:
(597, 7)
True
(597, 7)
(597, 7)
(312, 7)
0.11858974358974358 0.2403846153846154 312
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
True
(599, 7)
(599, 7)
(318, 7)
0.0031446540880503146 0.012578616352201259 318
df shap

C:\Users\User\AppData\Local\Continuum\anaconda3\envs\conda37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars


df shape final:
(599, 7)
True
(599, 7)
(599, 7)
(369, 7)
0.0 0.0027100271002710027 369
df shape before Lat Lon process:
(600, 4)
df shape after Lat Lon process:
(600, 4)
df shape final:
(599, 7)
True
(599, 7)
(599, 7)
(333, 7)
0.07807807807807808 0.14414414414414414 333


C:\Users\User\AppData\Local\Continuum\anaconda3\envs\conda37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in double_scalars


In [28]:
records.notnull().values.all()

False

In [29]:
records[records["Skewness"].isna()]

,Mean,Stdev,Over60,O60U120,Over120,Skewness,Kurtosis,Entropy,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9
3411,0.000096,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096,0.000096


In [30]:
labels.columns = ["Label"]
labels

,Label
0,1
1,0
2,1
3,0
4,1
5,0
6,1
7,0
8,1
9,0


In [129]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [142]:
norm_records = normalize(records)
norm_records
# records

,Mean,Stdev,Over60,O60U120,Over120,Skewness,Kurtosis,Entropy,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9
0,0.145505,0.191417,0.012617,0.025377,0.000000,0.326345,0.104499,0.682408,0.000794,0.027452,0.085648,0.101511,0.116216,0.149346,0.183605,0.234922,0.238319
1,0.714419,0.883772,0.598920,0.418985,0.555937,0.139932,0.004748,0.404345,0.264626,0.506220,0.585695,0.464856,0.467152,0.636224,1.000000,1.000000,1.000000
2,0.158509,0.237388,0.021223,0.028457,0.010069,0.365801,0.153686,0.822908,0.009839,0.043403,0.076443,0.111558,0.121438,0.149266,0.205312,0.262911,0.295188
3,0.818820,0.801329,0.741065,0.654905,0.591299,0.098016,0.006658,0.503054,0.516166,0.621832,0.699757,0.617908,0.628860,0.821033,1.000000,1.000000,1.000000
4,0.144767,0.115970,0.011228,0.022583,0.000000,0.272825,0.067659,0.740057,0.021830,0.078535,0.135081,0.125598,0.118405,0.153980,0.178427,0.217738,0.225408
5,0.887069,0.681102,0.896055,0.989011,0.575461,0.061093,0.018492,0.515395,0.779042,0.825105,0.924119,0.734570,0.717271,0.820496,1.000000,1.000000,1.000000
6,0.203506,0.315721,0.059540,0.079834,0.028248,0.426597,0.178579,0.854258,0.077022,0.121150,0.149106,0.127150,0.130796,0.178861,0.224917,0.280596,0.324507
7,0.702163,0.809838,0.637438,0.692308,0.417339,0.133345,0.009180,0.574555,0.294510,0.443817,0.594980,0.497318,0.515080,0.674047,0.789756,1.000000,1.000000
8,0.165085,0.170498,0.019900,0.040025,0.000000,0.307582,0.079670,0.770834,0.084584,0.097006,0.139577,0.116610,0.123976,0.149846,0.193251,0.249350,0.279539
9,0.540946,0.826052,0.453798,0.525497,0.274012,0.211353,0.014571,0.628125,0.208724,0.260499,0.316893,0.313927,0.320632,0.460803,0.545582,0.835455,1.000000


In [106]:
labels

,Label
0,1
1,0
2,1
3,0
4,1
5,0
6,1
7,0
8,1
9,0


In [31]:
np.savetxt('records.txt', records, delimiter=',')
np.savetxt('labels.txt', labels, delimiter=',')
# np.savetxt('normRecords.txt', norm_records, delimiter=',')

In [92]:
r = records["Mean"]
type(r)

pandas.core.series.Series

In [32]:
records

,Mean,Stdev,Over60,O60U120,Over120,Skewness,Kurtosis,Entropy,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9
0,0.003078,0.003131,0.002941,0.002941,0.000000,2.258956,8.236019,5.751367,0.000269,0.000760,0.001188,0.001597,0.002219,0.002801,0.003552,0.005031,0.006540
1,0.007645,0.010404,0.165517,0.065517,0.100000,1.670152,1.367946,5.333151,0.000489,0.000998,0.001407,0.001966,0.002758,0.003999,0.006309,0.011683,0.032240
2,0.003881,0.003859,0.007614,0.005076,0.002538,2.179733,9.354363,5.917910,0.000237,0.000712,0.001268,0.002058,0.003059,0.003922,0.004966,0.006200,0.008766
3,0.008080,0.009577,0.151751,0.073930,0.077821,1.696668,1.682462,5.315945,0.000903,0.001818,0.002457,0.003211,0.004104,0.005066,0.007287,0.013124,0.026197
4,0.003103,0.002808,0.002092,0.002092,0.000000,1.828245,4.573690,6.152776,0.000455,0.000837,0.001323,0.001853,0.002402,0.003047,0.003665,0.004657,0.006881
5,0.008272,0.009388,0.134211,0.052632,0.081579,1.733275,1.913344,5.660030,0.001124,0.001888,0.002535,0.003504,0.004653,0.005996,0.008065,0.011763,0.025691
6,0.004573,0.004401,0.020548,0.013699,0.006849,2.744150,12.468789,6.062034,0.000575,0.001397,0.002141,0.002819,0.003505,0.004300,0.005254,0.007057,0.009433
7,0.011256,0.010870,0.230189,0.105660,0.124528,1.047077,-0.272514,5.144316,0.000971,0.002296,0.003667,0.005070,0.006851,0.009714,0.013332,0.021087,0.033300
8,0.003201,0.003190,0.004902,0.004902,0.000000,2.226678,6.470550,5.997676,0.000591,0.001004,0.001298,0.001713,0.002192,0.002810,0.003508,0.004560,0.007268
9,0.007306,0.009356,0.136232,0.066667,0.069565,1.851075,2.276107,5.622462,0.000747,0.001334,0.001858,0.002543,0.003414,0.004464,0.006282,0.011106,0.023194
